# Data Science Ex 05 - Association Analysis

22.03.2020, Lukas Kretschmar (lukas.kretschmar@hsr.ch)

## Let's have some Fun with Market Basket Analysis!

In this notebook you are going to build and find frequent item sets.
And define rules which products are entangled with each other.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set()

## Introduction

### Preparing Data

As you've learned, data for the market basket analysis must be in a specific structure.
If not, you need to restructure it (see the end of the introduction).

For the introduction, let's assume that we already have data in the format we need.

In [ ]:
mba = pd.read_csv("./Demo_MarketBasket.csv")
print(f"Number of receipts: {len(mba)}")
mba.head(5)

### Importing the Modules

The needed algorithms are part of the *mlxtend* module.
With the following lines, we just import the algorithms that we need to do our analysis.

Anaconde does not contain the *mlextend* module by default and you have to download it with the following command in the `conda-prompt`.

```
conda install -c conda-forge mlxtend
```

And then you can import the methods from the module.

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

### Creating FI Sets

Reference: http://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/

We can now build our frequent item sets by calling the `apriori()` algorithm that we imported in the line above.
Some of the parameters offered by the algorithm are listed below.
There are some more, but not relevant for us.
All are optional besides the first one - we need data, obviously:
- `df` : `DataFrame` containing the data
- `min_support` (default: 0.5) : Minimal support the items have to fulfill to be included in the result
- `use_colnames` (default: False) : Column names are used instead of their indices in the item sets
- `max_len` (default: None) : Maximum number of itemsets returned

In [ ]:
fi_sets = apriori(mba, min_support=.05, use_colnames=True)
fi_sets

Having our frequent item sets, we can now build the association rules.
The `associatin_rules()` method offers the following parameters:
- `df` : `DataFrame` containing the frequent item sets
- `metric` (default: `"confidence"`) : Metric used to define if a rule is interesting. Possible values are:
    - `"support"`
    - `"confidence"`
    - `"lift"`
    - `"leverage"`
    - `"conviction"`
- `min_threshold` (default: 0.8) : Threashold the metric has to exceed to be considered interesting
- `support_only` (default: False) : Only computes support, and sets `metric` to "support"

In [ ]:
rules = association_rules(fi_sets, metric="confidence", min_threshold=.2)
rules

### Analysing Association Rules

In the `antecedents` column we find the left side of the rule (what was bought) and the `consequents` column contains the right side (what was also bought).
And since the rules are a `DataFrame`, we can filter, group and visualize the data with the techniques introduced in the last couple of weeks.

In [ ]:
rules[(rules["confidence"] >= .3) & (rules["lift"] >= 1.5)]

We see that milk was bought together with vegetables or yogurt more likely.

Another possibility is using visualization.

In [ ]:
fig, ax = plt.subplots() # We need this call, otherwiese the x-ticks won't show up (it's a bug in matplotlib -> https://github.com/pandas-dev/pandas/issues/10611)
rules.plot.scatter(ax=ax, x="confidence", y="lift", c="support", cmap="rainbow")

At the end, it's up to the data scientist to read and interpret the data properly, since domain-knowledge is a vital part of the analysing process.

### Nice2Know: Restructuring Data

While researching this topic, I noticed that there is no information available how you can preprocess your data to fit the algorithms needs.
Since preprocessing steps highly depend on the given data, it's no surprise there is no single one way to do so.

But I included here a step by step approach how you can change the data structure usually found to a format the `apriori()` algorithm can work with.
The problem with this algorithm is, that it expects data in a specific format.
And usually, data is available in a format like this:

In [ ]:
carts = pd.read_csv("./Demo_Groceries_Unstructured.csv")
carts.head(5)

As you can see, we have a row per shopping cart/receipt.
But as a result, we have many missing entries since the number of columns need to cover the largest cart.
And the given structure is not suitable for the algorithm.
Thus, we have to restructure the data so it can be used by the algorithm.

1. **Let's get rid of the *Item(s)* column, since we do not need it.
But we store its content to validate our new structure at the end.**

In [ ]:
oldSums = carts["Item(s)"]
carts = carts.drop("Item(s)", axis=1)
carts.head(1)

2. **Then we stack our data.
As a result, we get a multi-index series containing every carts item in a column.
We do this step to get rid of all the `NaN` entries in the data.**

In [ ]:
stackedCarts = carts.stack()
stackedCarts.head(10)

3. **By reseting the index, we get back a `DataFrame` containing the cart (level_0) and item (0).**

In [ ]:
stackedCarts = stackedCarts.reset_index()
stackedCarts.head(5)

4. **To introduce some readability, we change the column names and drop the obsolete column.**

In [ ]:
stackedCarts = stackedCarts.rename(columns={"level_0" : "Cart", 0 : "Item"}).drop("level_1",axis=1)
stackedCarts.head(5)

Sometimes, you already get data in a format like shown above.
So, the steps before are not necessary while preprocessing.

5. **We now add a new column, indicating that an item is in a cart.**

**Please note:** This step is only necessary when there is no amount information available.
Depending on the dataset you have, it's possible that there is already an amount asigned per item.
And then you don't need to add the numbers by yourself.

In [ ]:
stackedCarts["Amount"] = 1
stackedCarts.head(5)

6. **Now we create groups by *Cart* and *Item*, select the *Amount* and sum it.**

In [ ]:
newCarts = stackedCarts.groupby(["Cart", "Item"])["Amount"].sum()
newCarts.head(10)

This structure has quite a resemblance to the structure we had after step 2 where we stacked the dataset.

7. **We now basically do the reverse by unstacking our new data structure.**

In [ ]:
newCarts = newCarts.unstack()
newCarts.head(5)

8. **We just have to replace the missing values with 0.
And we can change the type to *int*.**

**FYI:** Sometimes the information indicating that an item was present is greater than 1.
In this case, you simply change the type first to `bool` and then to `int`.
Enforcing `bool` will assign `True` to every value other than `0`.

In [ ]:
newCarts = newCarts.fillna(0).astype(bool).astype(int)
newCarts.head(5)

And that's about it.

We now just have to check that our values are still matching the original data.
An easy check is the number of items per cart.

In [ ]:
newSums = newCarts.sum(axis=1)
newSums

In [ ]:
(newSums - oldSums).any()

`False` indicates that there are no differences.
For us, this is a good enough indicator that we didn't change the information contained in the data at all.

## Exercises

### Ex01 - Apriori on a simple Dataset

Load the file **Ex05_01_Data.csv** and list the first 5 lines.
You may have to specify the separator `sep` used to get a correct `DataFrame`.

You should already be familiar with this data.
But as you can see, some values are missing and there are `X` instead of `1`.
Correct this and show the first 5 lines again.

Create the frequent items set with a `min_support >= .3`.
And show the whole itemset.

Find the rules that have a `confidence >= .85` and list them.

#### Solution

In [ ]:
# %load ./Ex05_01_Sol.py

### Ex02 - Apriori on a large Dataset

Load the file **Ex05_02_Data.csv** and list the first couple of lines.

How many shopping carts are in the dataset?

Find the frequent item sets with a `min_support >= .3`.

Build the rules with `confidence >= .5`.

How many rules did you find?

Sort the rules by their `lift` with the most promising rule on top.

#### Solution

In [ ]:
# %load ./Ex05_02_Sol.py

### Ex03 - Processing Data

Load the file **Ex05_03_Data.csv** and show the first few lines.

As you can see, the data is not structured the way we need it to run the apriori algorithm.
It's now your job to process the data.
The goal is that you can use the result of this exercise as the input of the next one (don't worry, the next exercise comes with it's own input data if you cannot accomplish this one).

As you see, the data is structured like **Step 5** in the introduction.
Thus, you can go on with **Steps 6 - 8**.
But compared to the intro, you now have also to include the *Country* into the grouping.

Compared to the example in the introduction, we have here a multi-index.
But that shouldn't bother us.
It's actually quite useful when filtering for one country.

If you want, you can store your `DataFrame` with `to_csv(filename)`[(Reference)](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html) and reload the file in the next exercise.
If you save your data, it could be handy to replace every `0` with `np.nan` again or just don't do *Step 8*.
This way, the output file needs 50% less storage capacity.

Or you can simple reuse the variable containing your `DataFrme` in the next exercise.

In [ ]:
# Code for storing file comes here - if you want

#### Solution

In [ ]:
# %load ./Ex05_03_Sol.py

### Ex04 - Apriori on your Dataset

The idea is that you use the processed data from the previous exercise.
If you were not able to complete the preprocessing, you can use **Ex05_04_Data.csv**.
This file contains the same data that you would have gotten from the preprocessing exercise.

If you start this exercise from a file (the provided one or the one you created in the previous exercise), you need to do the following steps (otherwise, you can skip these steps):

1. Load the content of the file.

2. Set *Country* and *InvoiceNo* as index with `set_index()`.

3. Fill the missing data (basically do step 8 from the introdcution).

**From here, the exercise is the same - for those starting from a file or taking the variable from the exercise above.**

First we will have a look at the information from *Switzerland*.
Thus, just select the invoices from `Switzerland` by using `df.loc["Switzerland"]` (`df` stands for your variable).

How many invocies are there?

Build the frequent item sets with a `min_support >= .1`.
And show all the rules with a `confidence >= .7` and order them by their `lift` (best on top).

How many rules did you find?

Since these retail information are from a UK based online store, there are much more invoivces from the UK.
So get them.

How many invoices from the UK are there?

Now, let's find the frequent item sets (use the same parameters from above).

Are there any frequent item sets? - Usually, the larger you datasets are, the lower you have to set the support threshold to even get results.
Thus, lower the `min_support` to `.03` and build the rules.

Are we surprised that these rules were found?

#### Solution

In [ ]:
# %load ./Ex05_04_Sol.py